In [1]:
import os
import shutil

In [9]:
"""
A sample output:


#!/bin/bash
#SBATCH --array=0-1
#SBATCH --mem=2048M
#SBATCH -t 1-00:00
#SBATCH -o s_sara_%job.out
#SBATCH --job-name=p75_single_SARA

source ~/unity_ml/bin/activate
python Test_Simulator.py True SARA ddpg 5 $SLURM_ARRAY_TASK_ID

"""

'\nA sample output:\n\n\n#!/bin/bash\n#SBATCH --array=0-1\n#SBATCH --mem=2048M\n#SBATCH -t 1-00:00\n#SBATCH -o s_sara_%job.out\n#SBATCH --job-name=p75_single_SARA\n\nsource ~/unity_ml/bin/activate\npython Test_Simulator.py True SARA ddpg 5 $SLURM_ARRAY_TASK_ID\n\n'

In [14]:
# Constants
header = "#!/bin/bash"
modes = ["PLA","SARA","Random"]
memories = [1800,2048,875]

# Variable
time = "1-00:00"
n_jobs = 2

n_visitor = 1
visitor_epsilon = 0
attr_model = "sequence"
algorithm = "ddpg" # td3, ppo

intervals = ["one","point75","point5","point25","point125"]

output_dir = os.path.join(os.path.abspath(".."),"SHARCNET","shell_scripts","job")

In [15]:
# clean up folder
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)

In [16]:
os.mkdir(output_dir)

In [17]:
# Write files

for itv in intervals:
    output_subdir = os.path.join(output_dir,itv)
    os.mkdir(output_subdir)
    shellscript_names = []
    
    for mode, memory in zip(modes,memories):
        visitor_mode = "single" if n_visitor == 1 else "multi"
        filename = "s_"+visitor_mode+"_"+mode.lower()+".sh"
        shellscript_names.append(filename)
        with open(os.path.join(output_subdir,filename), 'w',  newline='\n') as shellfile:
            shellfile.write(header)
            shellfile.write("\n")
            shellfile.write("#SBATCH --array=0-{}".format(n_jobs-1))
            shellfile.write("\n")
            shellfile.write("#SBATCH --mem={}M".format(memory))
            shellfile.write("\n")
            shellfile.write("#SBATCH -t {}".format(time))
            shellfile.write("\n")
            shellfile.write("#SBATCH -o s_{}_%job.out".format(mode.lower()))
            shellfile.write("\n")
            shellfile.write("#SBATCH --job-name={}_{}_{}_{}".format(itv,visitor_mode,visitor_epsilon,mode.lower()))
            shellfile.write("\n")

            shellfile.write("source ~/unity_ml/bin/activate")
            shellfile.write("\n")
            shellfile.write("python Test_Simulator.py True {} {} {} {} {} $SLURM_ARRAY_TASK_ID".format(mode, algorithm, n_visitor, visitor_epsilon, attr_model))
    
    with open(os.path.join(output_subdir,"submit.sh"), 'w',  newline='\n') as shellfile:
        shellfile.write(header)
        shellfile.write("\n")
        for mode, name in zip(modes,shellscript_names):       
            shellfile.write("echo Submitting {} job".format(mode) )
            shellfile.write("\n")
            shellfile.write("sbatch {}".format(name))
            shellfile.write("\n")

In [18]:
output_dir

'F:\\unity_simulator\\unity_simulator\\SHARCNET\\shell_scripts\\job'